In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importation des données

In [2]:
# Importation du dataset issu de vgchartz.com (obtenu sur kaggle).
df = pd.read_csv('vgsales.csv')


In [3]:
# Suppression des entrées contenant des valeurs manquantes et suppression de la variable Rank.
df = df.dropna()
df = df.drop(['Rank'], axis = 1)

In [4]:
# Concaténation des données issues de notre webscrapping sur le site metacritic.

# YANN : remplacer par l'importation d'un fichier déjà concaténé

df_ratings1 = pd.read_csv('metacritic0-25', usecols=['game', 'platform', 'rating'])
df_ratings2 = pd.read_csv('metacritic25-50.csv', usecols=['game', 'platform', 'rating'])
df_ratings3 = pd.read_csv('metacritic50-75.csv', usecols=['game', 'platform', 'rating'])
df_ratings4 = pd.read_csv('metacritic75-100.csv', usecols=['game', 'platform', 'rating'])
df_ratings5 = pd.read_csv('metacritic100-125.csv', usecols=['game', 'platform', 'rating'])
df_ratings6 = pd.read_csv('metacritic125-150.csv', usecols=['game', 'platform', 'rating'])
df_ratings7 = pd.read_csv('metacritic150-175.csv', usecols=['game', 'platform', 'rating'])
df_ratings8 = pd.read_csv('metacritic175-186.csv', usecols=['game', 'platform', 'rating'])
df_ratings = pd.concat([df_ratings1, df_ratings2, df_ratings3, df_ratings4, df_ratings5, df_ratings6, df_ratings7, df_ratings8],ignore_index = True)

# Jointure

In [14]:
# Modification des noms des colonnes pour que les variables utilisées comme clés pour le merge coincident entre df et df_ratings.
df_ratings = df_ratings.rename(columns = {'game' : 'Name', 'platform' : 'Platform'})
# Modification des noms des platformes (variable Platform) qui étaient différentes entre df et df_ratings
df_ratings = df_ratings.replace(['PlayStation 4', 'Xbox 360', 'PlayStation 2', 'PlayStation 3', 'Xbox One', 'Xbox', 'GameCube', 
                                 'Game Boy Advance', 'PlayStation Vita', 'PlayStation', 'Wii U', 'Dreamcast', 'Nintendo 64'], 
                                ['PS4', 'X360', 'PS2', 'PS3', 'XOne', 'XB', 'GC', 'GBA', 'PSV', 'PS', 'WiiU', 'DC', 'N64'])

In [15]:
# Jointure des deux Dataframes avec merge. Certains jeux sont sortis sur plusieurs plateformes. Pour chaque jeux on a donc 
# une entrée par plateforme du jeux. Il faut donc faire une jointure sur deux variables : Name et Plateform.
df_mer = df.merge(right = df_ratings, on = ['Name', 'Platform'])

In [16]:
# Comparaison des tailles des dataframes
# Fusion des 2 bases de donnée
print("taille Dataframe principale: ", df.shape)
print("taille Dataframe webscrappée: ", df_ratings.shape)


print("taille BDD mergée: ", df_mer.shape)
df_mer.head()

taille Dataframe principale:  (16291, 10)
taille Dataframe webscrappée:  (18600, 3)
taille BDD mergée:  (6808, 11)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,76
1,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,82
2,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,80
3,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01,89
4,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02,58


La majorité des données des dataframes initiales ne sont pas conservées par la jointure. Une première explication peut être donnée : Le dataset obtenu sur kaggle ne contient pas les jeux vidéos sortis après 2017, alors que les données webscrappé sur métacritiques sont à jour. Les jeux sortis entre 2017 et 2021 sont donc forcément exclues par la jointure. Cela ne suffit cependant pas à expliquer l'ampleur des données no conservées par la jointure.

In [17]:
# Selection des données issues du dataset de kaggle qui n'ont pas été conservées par la jointure.
df_out = df[-df.Name.isin(df_mer.Name)]

In [18]:
# Affichage des 20 premières données (classées par vente)
df_out[0:20]


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31
10,Nintendogs,DS,2005.0,Simulation,Nintendo,9.07,11.00,1.93,2.75,24.76
12,Pokemon Gold/Pokemon Silver,GB,1999.0,Role-Playing,Nintendo,9.00,6.18,7.20,0.71,23.10
18,Super Mario World,SNES,1990.0,Platform,Nintendo,12.78,3.75,3.54,0.55,20.61
19,Brain Age: Train Your Brain in Minutes a Day,DS,2005.0,Misc,Nintendo,4.75,9.26,4.16,2.05,20.22
20,Pokemon Diamond/Pokemon Pearl,DS,2006.0,Role-Playing,Nintendo,6.42,4.52,6.04,1.37,18.36
21,Super Mario Land,GB,1989.0,Platform,Nintendo,10.83,2.71,4.18,0.42,18.14


Beaucoup de jeux anciens n'ont pas été conservé à la jointure, y compris certains qui ont fait beaucoup de ventes. Nous ne pouvons pas traiter chaque cas, mais nous pouvons regarder les 5 premiers pour essayer de comprendre pourquoi le jeux n'ont pa été conservés.

Super Mario Bros : Le site metacritic ne contient pas de fiche de ce jeux, mais seulement de ses remakes et suites sur des consoles plus récentes.

Pokemon Red/Pokemon Blue : Idem

Tetris : Idem

Duck Hunt : Idem

Nitendogs : Ce jeux est sorti en plusieurs versions :  Nintendogs: Chihuahua & Friends, Nintendogs: Dalmatian & Friends... qui ont chacune une fiche sur metacritic mais sont rassemblées en une seule entrée dans le dataset de kaggle

Dans la suite de la liste, on voit beaucoup de jeux pokemon qui sont sortis en plusieurs version. Dans le dataset de kaggle, ils sont rassemblés en un seul jeu (exemple : Pokemon Gold/Pokemon Silver) alors que sur metacritic, ils sont séparés en deux jeux (Pokemon Gold et Pokemon Silver).

L'absence d'ancien jeux dans les données de metacritic et les problèmes de version de jeux (Nintendogs, Pokemon) semblent expliquer presque tous les 20 cas affichés. Nous ne voyons pas comment nous pourrions corriger ces problème de manière auomatisée. Nous continuons donc notre projet avec le dataframe mergé tel quel.

In [19]:
# Exportation du dataframe mergé
df_mer.to_csv('df_merged.csv')